# Lab 4: Extending Logistic Regression

### Data Mangling:

Mangling of the CSV:
- We first removed the imdb link from the csv because we knew we would never need to use that (**Note: this was the only feature removed from the csv**)
- We then went through and deleted all of the movies that were made in another country (foriegn films) we did this because we wanted to just look at American films, also because the currency units for those countries (for budget and gross) were in native currency units, not USD, and with changing exchange rates, it's not very easy to compare across countries.
- We then went through and converted all 0 values for gross, movie_facebook_likes, and director_facebook_likes to a blank value in the csv (so that it is read in as NaN by pandas), this is so that we cna more easily impute values later. Note: according to the description on the kaggle entry, because of the way the data was scraped, some movies had missing data. The Python scraper just made these values into a 0 instead of NaN.
- We then removed all movies with an undefined gross. Being the feature we are trying to predict, we should not be imputing values for gross to train our model. That will basically reduce our model to an imputation algorithm...
- We then removed all movies that were made before 1935. We did this because there were only a handful of movies ranging from 1915 to 1935, the way we are classifying budget (described below) would not work with a small sample of movies from that time period. We could have cut this number at a different year (say 1960), but we didn't want to exclude such classics as "Bambi" or "Gone With the Wind"

Mangling of the Data:
- After the above steps, we made more edits to the data using pandas. First, we removed features that we thought would be un-useful to our prediction algorithm. We removed all features concerning facebook likes. We did this because a significant portion of the movies in the training set debuted before facebook was invented and widely adopted. While some of these movies have received retroactive "likes" on facebook, only the most famous classics received a substantial amount of retraoctive "likes". Most lesser known films received very low amounts of "likes" (presumably because modern movie watchers don't really care to search for lesser known movies on facebook, or because the movie doesn't have a facebook). For this reason we decided to remove movie_facebook_likes
- Likewise, we removed the other "likes" for the same reasons as above. For example, the esteemed director George Lucas has a total of 0 "likes" between all of his films. This feature obviously would not help us predict the profitability of movies.
- We also removed irrelevant information such as aspect_ratio, language, and country. Because we deleted all foreign films the country will always be USA. A simple filter of the data reveals that there are no more than 20 movies made in the US that use a language other than English, therefore there is not enough data to use language as training feature. However, we did not delete the movies in a different language, because most of them were famous films such as *Letters from Iwo Jima* and *The Kite Runner*. We still count them as a valuable part of the dataset, just don't find the language of particular value. Lastly, we removed aspect_ratio because that seems to be unimportant for predicting the success of a movie.

In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv("movie_metadata.csv")
for x in ['movie_facebook_likes', 'director_facebook_likes', 'actor_2_facebook_likes', 
          'actor_1_facebook_likes','actor_3_facebook_likes', 'cast_total_facebook_likes',
          'aspect_ratio', 'language', 'country']:
    if x in df:
        del df[x]
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3232 entries, 0 to 3231
Data columns (total 18 columns):
color                     3231 non-null object
director_name             3232 non-null object
num_critic_for_reviews    3229 non-null float64
duration                  3231 non-null float64
actor_2_name              3229 non-null object
gross                     3232 non-null int64
genres                    3232 non-null object
actor_1_name              3230 non-null object
movie_title               3232 non-null object
num_voted_users           3232 non-null int64
actor_3_name              3225 non-null object
facenumber_in_poster      3226 non-null float64
plot_keywords             3208 non-null object
num_user_for_reviews      3231 non-null float64
content_rating            3206 non-null object
budget                    3071 non-null float64
title_year                3232 non-null int64
imdb_score                3232 non-null float64
dtypes: float64(6), int64(3), object(9)
memo

In [38]:
# Tamper with the groupings to improve imputations? How do we improve how many values get imputed?
df_grouped = df.groupby(by=['director_name'])
# director_name adds about 50 rows (imputes about 50 rows and then deletes about 100)

In [39]:
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))
col_deleted = list( set(df.columns) - set(df_imputed.columns)) #in case the median op deleted columns
df_imputed[col_deleted] = df[col_deleted]

print(df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3232 entries, 0 to 3231
Data columns (total 18 columns):
num_critic_for_reviews    3229 non-null float64
duration                  3231 non-null float64
gross                     3232 non-null int64
num_voted_users           3232 non-null int64
facenumber_in_poster      3232 non-null float64
num_user_for_reviews      3231 non-null float64
budget                    3159 non-null float64
title_year                3232 non-null int64
imdb_score                3232 non-null float64
color                     3231 non-null object
movie_title               3232 non-null object
genres                    3232 non-null object
plot_keywords             3208 non-null object
actor_2_name              3229 non-null object
director_name             3232 non-null object
content_rating            3206 non-null object
actor_3_name              3225 non-null object
actor_1_name              3230 non-null object
dtypes: float64(6), int64(3), object(9)
memo

In [40]:
# drop rows that still have missing values after imputation
df_imputed.dropna(inplace=True)
print (df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3119 entries, 0 to 3230
Data columns (total 18 columns):
num_critic_for_reviews    3119 non-null float64
duration                  3119 non-null float64
gross                     3119 non-null int64
num_voted_users           3119 non-null int64
facenumber_in_poster      3119 non-null float64
num_user_for_reviews      3119 non-null float64
budget                    3119 non-null float64
title_year                3119 non-null int64
imdb_score                3119 non-null float64
color                     3119 non-null object
movie_title               3119 non-null object
genres                    3119 non-null object
plot_keywords             3119 non-null object
actor_2_name              3119 non-null object
director_name             3119 non-null object
content_rating            3119 non-null object
actor_3_name              3119 non-null object
actor_1_name              3119 non-null object
dtypes: float64(6), int64(3), object(9)
memo

#### Split Up the Data Now

In [47]:
from sklearn import model_selection
train_set, test_set = model_selection.train_test_split(df_imputed, test_size=0.2, train_size=0.8, random_state= 101)
print(len(train_set), len(test_set))
## http://localhost:8888/notebooks/fork/MachineLearningNotebooks/05.%20Logistic%20Regression.ipynb#Training-and-Testing-Split

2495 624


# WHAT IS CROSS VALIDATION???

Answer...It's performed automatically above^^ with the train_test_split, we just don't have 3 iterations to use later. Which is fine for now, Larson just said we will be doing it in later labs.

___

# But First... A Super Cool Custom Regression 

**Requirement**: [20 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template used in the course. You should add the following functionality to the logistic regression classifier:

In [55]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.special import expit

ds = load_iris()
X = ds.data
y = (ds.target>1).astype(np.int) # make problem binary

In [56]:
%%time
# from last time, our logistic regression algorithm is given by (including everything we previously had):
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += 2 * self.w_[1:] * self.C
        
        return gradient
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

blr = BinaryLogisticRegression(eta=0.1,iterations=500,C=0.001)

blr.fit(X,y)
print(blr)

yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Binary Logistic Regression Object with coefficients:
[[-0.79466917]
 [-1.65729122]
 [-1.49602257]
 [ 2.45160555]
 [ 2.10998729]]
Accuracy of:  0.98
Wall time: 29.1 ms


Wall time: 0 ns


In [51]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T  ##create a matric of weights for the different classes
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier  ##go through the list of classifiers and get predictions for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

_____
**Requirement**: Ability to choose optimization technique when class is instantiated: either steepest descent, stochastic gradient descent, or Newton's method. 